# **GIT CLONE**

https://github.com/ThaiMinhLam/MRC_2.0

In [1]:
!git clone https://github.com/ThaiMinhLam/MRC_2.0

Cloning into 'MRC_2.0'...
remote: Enumerating objects: 273, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 273 (delta 93), reused 1 (delta 1), pack-reused 121 (from 1)
Receiving objects: 100% (273/273), 64.41 MiB | 18.04 MiB/s, done.
Resolving deltas: 100% (137/137), done.


In [2]:
# hàm đọc file json
import json

def read_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

sgk = read_json_file("/content/MRC_2.0/sgk_social_subject_new.json")
qas = read_json_file("/content/MRC_2.0/QAs_final_upd_4.json")

# **FORMAT DATA INTO DATAFRAME**

In [15]:
import pandas as pd

df = pd.DataFrame({"question_url":[],
                   "question_code":[],
                   "question":[],
                   "A":[],
                   "B":[],
                   "C":[],
                   "D":[],
                   "context":[],
                   "correct_answer":[],
                   "explanation":[]
                   })

for code, value_code in qas.items():
    question_url = value_code["question_url"]
    question_code = code
    question = value_code["question"]
    A = value_code["answer_options"][0]
    B = value_code["answer_options"][1]
    C = value_code["answer_options"][2]
    D = value_code["answer_options"][3]
    correct_answer = value_code['correct_answer']
    explanation = value_code["explanation"]
    lesson_chapter = sgk[code[0]][code[1:3]]
    for chapter, lesson in lesson_chapter.items():
        for lesson_code, context_name in lesson.items():
            if(lesson_code == code[3:6]):
                context = context_name["context"]
                break
    df.loc[len(df)] = [question_url, question_code, question, A, B, C, D, context, correct_answer, explanation]

In [16]:
df.head()

,question_url,question_code,question,A,B,C,D,context,correct_answer,explanation
0,https://vietjack.me/trac-nghiem-tong-quan-van-...,V10B0100000,Văn học Việt Nam do các bộ phận văn học nào hợ...,A. Văn học dân gian và văn học trung đại,B. Văn học trung đại và văn học hiện đại,C. Văn học dân gian và văn học viết,D. Văn học hiện đại và văn học dân gian,"Trải qua hàng ngàn năm lịch sử, nhân dân Việt ...",C,Văn học Việt Nam bao gồm các sáng tác ngôn từ ...
1,https://vietjack.me/trac-nghiem-tong-quan-van-...,V10B0100001,Đặc điểm cơ bản nhất của thể loại truyền kỳ là...,"A. Dùng yếu tố hoang đường, kỳ ảo làm phương t...",B. Lựa chọn những sự kiện lịch sử của quá khứ ...,C. Có giá trị hiện thực và nhân đạo cao,D. Truyền bá niềm tin vào những điều tốt đẹp c...,"Trải qua hàng ngàn năm lịch sử, nhân dân Việt ...",A,Đặc điểm cơ bản nhất của thể loại truyền kỳ là...
2,https://vietjack.me/trac-nghiem-tong-quan-van-...,V10B0100002,Chữ Nôm là loại chữ nào?,A. Loại chữ người Việt cổ tự sáng tạo ra để gh...,"B. Loại chữ cổ của người Việt, dựa vào chữ Hán...","C. Loại chữ cổ của người Việt, dùng chữ Hán để...","D. Loại chữ của người Việt cổ, dùng chữ Hán để...","Trải qua hàng ngàn năm lịch sử, nhân dân Việt ...",B,"Chữ Nôm là loại chữ cổ của người Việt, dựa vào..."
3,https://vietjack.me/trac-nghiem-tong-quan-van-...,V10B0100003,Thời kì văn học Việt Nam từ đầu thế kỉ XX - Cá...,A. Lịch sử Việt Nam có sự thay đổi lớn,B. Nền văn học Việt Nam có sự xuất hiện của cá...,C. Tư tưởng văn hóa Phương Đông du nhập,D. Cơ cấu xã hội Việt Nam thay đổi,"Trải qua hàng ngàn năm lịch sử, nhân dân Việt ...",A,Thời kì văn học Việt Nam từ đầu thế kỉ XX - Cá...
4,https://vietjack.me/trac-nghiem-tong-quan-van-...,V10B0100004,Chữ Nôm và chữ quốc ngữ giống nhau ở điểm nào?,A. Đều vay mượn từ Trung Hoa,B. Đều do nho sĩ sáng tạo ra,C. Đều ghi được âm tiếng Việt,D. Đều dùng chữ cái La-tinh,"Trải qua hàng ngàn năm lịch sử, nhân dân Việt ...",C,Chữ Nôm và chữ quốc ngữ giống nhau ở điểm đều ...


In [9]:
!pip install torch

In [10]:
!pip install transformers

In [11]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# **EVALUATE LLM**

In [22]:
#calling model name with its tokenizer
model_name = "bhavikardeshna/xlm-roberta-base-vietnamese"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/756 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [23]:
#using pipeline method
qa = pipeline('question-answering', model=model_name, tokenizer = model_name)

## test few records for fun

In [58]:
input_question = df[df["question_code"] == "D11B0800949"]["question"]
input_context = df[df["question_code"] == "D11B0800949"]["context"]

In [61]:
input_question

,question
5642,Ngành công nghiệp nào được coi là ngành kinh t...


In [67]:
input_context.iloc[0]

'diện tích : 17,1 triệu $\\mathrm{km}^{2}$. dân số : 143 triệu người (năm 2005). thủ đô : mát-xcơ-va. i – vị trí địa lí và lãnh thổ. lb nga có diện tích lớn nhất thế giới, nằm ở cả hai châu lục á, âu. lãnh thổ trải dài trên phần lớn đồng bằng đông âu và toàn bộ phần bắc á. lb nga có đường biên giới xấp xỉ chiều dài xích đạo. đất nước trải ra trên 11 múi giờ, giáp với 14 nước (trong đó có 8 nước thuộc liên xô trước đây). riêng tỉnh ca-li-nin-grát nằm biệt lập ở phía tây, giáp với ba lan và lít-va. lb nga có đường bờ biển dài. phía bắc giáp bắc băng dương, phía đông giáp thái bình dương, phía tây và tây nam giáp biển ban tích, biển đen, biển ca-xpi. những vùng biển rộng lớn này có giá trị nhiều mặt đối với sự phát triển kinh tế – xã hội của đất nước. ii – điều kiện tự nhiên. – địa hình lb nga cao ở phía đông, thấp dần về phía tây. dòng sông ê-nít-xây chia lb nga ra thành 2 phần rõ rệt :. + phần phía tây. đại bộ phận là đồng bằng (đồng bằng đông âu, đồng bằng tây xi-bia) và vùng trũng. đồ

In [72]:
output0 = qa(input_question.iloc[0], input_context.iloc[0])

In [73]:
output0

{'score': 0.8814488053321838,
 'start': 6286,
 'end': 6316,
 'answer': ' công nghiệp khai thác dầu khí'}

## test real

In [75]:
df.shape

(13109, 10)

In [ ]:
for i in range (len(df)):
    df.iloc[i]["answer_generate"] = qa(df.iloc[i]["question"], df.iloc[i]["context"])